In [1]:
import numpy as np
import pandas as pd

In [52]:
columns_movie = ['genres','belongs_to_collection','id','title','popularity','vote_average','vote_count','tagline','overview']
movies_df_ori = pd.read_csv('./data/movies_metadata.csv', usecols=columns_movie)
movies_df_ori = movies_df_ori.drop([19730, 29503, 35587])
movies_df_ori['popularity'] = movies_df_ori['popularity'].astype('float')

/home/abdev/projects/mlai-projects/movie-recommendation/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [54]:
movies_df = movies_df_ori.loc[ (movies_df_ori['vote_count']>30) & (movies_df_ori['popularity']>4) ].reset_index()
print(movies_df.shape)
# movies_df.describe()

(9262, 10)


# using tags and overview

In [60]:
movies_df['tagline'] = movies_df['tagline'].fillna('')
movies_df['overview'] = movies_df['overview'].fillna('')
movies_df['description'] = movies_df['tagline'] + movies_df['overview']
movies_df['description'] = movies_df['description'].fillna('')

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [63]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

In [64]:
desription_matrix = tf.fit_transform(movies_df['description'])

In [69]:
cosine_sim = linear_kernel(desription_matrix, desription_matrix)

In [70]:
movies_df.reset_index()
indices = pd.Series(movies_df.index, index=movies_df['title'])

In [83]:
human_friendly_cols = ['id','title','overview','tagline']

In [84]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return movies_df.iloc[movie_indices][human_friendly_cols]

In [91]:
movies_df[ movies_df['title'].str.contains('Infinity War') ][human_friendly_cols]

,id,title,overview,tagline


In [121]:
get_recommendations('The Dark Knight').head(10)

,id,title,overview,tagline
6191,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,The Legend Ends
83,414,Batman Forever,The Dark Knight of Gotham City confronts a das...,"Courage now, truth always..."
725,364,Batman Returns,"Having defeated the Joker, Batman now faces th...","The Bat, the Cat, the Penguin."
6141,69735,Batman: Year One,Two men come to Gotham City: Bruce Wayne after...,A merciless crime turns a man into an outlaw.
329,268,Batman,The Dark Knight of Gotham City begins his war ...,Have you ever danced with the devil in the pal...
5709,40662,Batman: Under the Red Hood,Batman faces his ultimate challenge as the mys...,Dare to Look Beneath the Hood.
6639,142061,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that Th...,Justice Returns... Vengeance Returns... Redemp...
8995,324849,The Lego Batman Movie,In the irreverent spirit of fun that made “The...,Always be yourself. Unless you can be Batman.
7588,20077,Batman vs Dracula,Gotham City is terrorized not only by recent e...,There's another bat in Gotham...
3842,16234,Batman Beyond: Return of the Joker,"The Joker is back with a vengeance, and Gotham...","This Time, The Joker Is Wild"


In [107]:
tmp_df = pd.read_csv('./data/movies_metadata.csv', usecols=['id','title','release_date'])

In [118]:
tmp_df.release_date.drop_duplicates(inplace=True)
tmp_df.dropna(inplace=True)

In [119]:
tmp_df.sort_values(by='release_date')

,id,release_date,title
34940,315946,1874-12-09,Passage of Venus
34937,194079,1878-06-14,Sallie Gardner at a Gallop
41602,426903,1883-11-19,Buffalo Running
34933,159897,1887-08-18,Man Walking Around a Corner
34934,96882,1888-01-01,Accordion Player
...,...,...,...
44535,412059,2018-04-04,Mobile Homes
38130,332283,2018-04-25,Mary Shelley
30402,38700,2018-11-07,Bad Boys for Life
38885,299782,2018-12-31,The Other Side of the Wind
